# Event Impact Experiment

本 notebook 依據 README 的實驗設計：
- 以「不含事件特徵」的基線模型學習常規結構（趨勢、季節、週期、假日）。
- 產生 out-of-sample 殘差並以穩健閾值定義「大殘差日」。
- 將大殘差日與事件日對齊，進行統計檢定與 AUC 評估。
- 視覺化殘差與事件窗口。

注意：下方舊有 Spark 相關單元為先前草稿，請以本區新增的 CSV 流程為準。


## Step 0 — 環境與套件

說明：載入分析所需套件、評估指標與工具函式。程式碼僅匯入與輔助方法，無資料處理。


## Step 1 — 讀取資料與基本檢查

說明：從 CSV 讀取 `filtered_df.csv`，處理日期型別與基本欄位型別，做最小化的 EDA（日期範圍與地點數）。


## Step 2 — 篩選地點與鎖定年度

說明：選擇分析地點（可用 `ACTIVE_LOCATION=None` 代表全部），鎖定 2024 年資料。此步驟不處理事件特徵。


## Step 3 — 定義基線特徵（排除事件）

說明：挑選只含一般/週期/假日的特徵，不包含任何事件欄位，避免洩漏。


## Step 4 — 時間切分（訓練/驗證）

說明：以 2024 年最後 28 天作為驗證集，其他為訓練。確保是 time-based split。


## Step 5 — 基線模型（不含事件）

說明：建置前處理（類別 One-Hot、數值標準化）與隨機森林回歸模型，訓練並在驗證集上評估 MAE/RMSE/sMAPE。


## Step 6 — 殘差與大殘差日

說明：計算殘差與絕對殘差，使用 MAD 建立穩健閾值，標記大殘差日。


## Step 7 — 事件對齊與統計檢定

說明：以事件旗標（類別非 'None' 或數值 > 0）與大殘差日建立列聯表，若可行則使用 Fisher 檢定，並以 |殘差| 作為分數計算 ROC-AUC。


## Step 8 — 視覺化

說明：繪製殘差時間序列，標示事件日與大殘差日，輔助直觀檢視事件影響。


## Step 9 — 多地點分析

說明：針對所有地點各自建立基線模型與驗證，計算殘差與事件對齊，輸出以顯著性（Fisher p 值）與 AUC 排序的摘要表。


## Step 10 — 匯出原始 + 預測表

說明：將 2024 年訓練與驗證資料合併，輸出包含原始欄位、預測值、殘差與分割標記的對照表。


In [ ]:
# Imports
import os
import math
import json
import numpy as np
import pandas as pd
from datetime import timedelta

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, roc_auc_score

try:
    from scipy.stats import fisher_exact
    SCIPY_AVAILABLE = True
except Exception:
    SCIPY_AVAILABLE = False

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

pd.set_option('display.max_columns', 200)

# Metrics helpers

def rmse(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))


def smape(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = (np.abs(y_true) + np.abs(y_pred))
    denom[denom == 0] = 1.0
    return 100.0 * np.mean(2.0 * np.abs(y_pred - y_true) / denom)


def mad(x):
    med = np.median(x)
    return np.median(np.abs(x - med))


In [ ]:
# Load data from CSV (replace legacy Spark flow)
CSV_PATH = "/Users/katadriano/Documents/ml_test_space/filtered_df.csv"
assert os.path.exists(CSV_PATH), f"CSV not found: {CSV_PATH}"

raw_df = pd.read_csv(CSV_PATH)

# Parse dates and basic types
raw_df['FT_CalendarDate'] = pd.to_datetime(raw_df['FT_CalendarDate'])
raw_df = raw_df.sort_values(['Location', 'FT_CalendarDate']).reset_index(drop=True)

# Ensure numeric columns are numeric
numeric_cols_guess = [
    'DayOfWeek','Month','isWeekend','isMacauHoliday','isChinaHoliday',
    'consecutiveChina','consecutiveMacau','DayOfMonth','Year','is_weekend',
    'is_month_start','EventSize_MinMax','days_to_holiday','holiday_weight',
    'holiday_decay','weekendHoliday','Actual'
]
for c in numeric_cols_guess:
    if c in raw_df.columns:
        raw_df[c] = pd.to_numeric(raw_df[c], errors='coerce')

# Event category as string/categorical
if 'EventSize_Category' in raw_df.columns:
    raw_df['EventSize_Category'] = raw_df['EventSize_Category'].astype(str)

# Basic EDA head
display(raw_df.head(5))
print("Date range:", raw_df['FT_CalendarDate'].min(), '->', raw_df['FT_CalendarDate'].max())
print("Locations:", raw_df['Location'].nunique())


In [ ]:
# Select one active location (or set to None to use all)
ACTIVE_LOCATION = "Chiu Chow - Pak Loh - GM"  # set to None to use all locations

if ACTIVE_LOCATION is not None:
    df = raw_df.loc[raw_df['Location'] == ACTIVE_LOCATION].copy()
else:
    df = raw_df.copy()

# Restrict to year 2024 for primary training/validation
mask_2024 = df['Year'] == 2024
df_2024 = df.loc[mask_2024].copy()
print(f"Records in 2024: {len(df_2024)} for location {ACTIVE_LOCATION}")

display(df_2024[['FT_CalendarDate','Location','Actual']].head(3))


In [ ]:
# Define baseline features (exclude event-related features)
ALL_FEATURES = [
    'DayOfWeek','Month','isWeekend','isMacauHoliday','isChinaHoliday',
    'consecutiveChina','consecutiveMacau','DayOfMonth','Year','is_weekend',
    'is_month_start','days_to_holiday','holiday_weight','holiday_decay','weekendHoliday'
]
EVENT_FEATURES = ['EventSize_MinMax','EventSize_Category']
TARGET = 'Actual'

# Keep only columns that exist
BASELINE_FEATURES = [c for c in ALL_FEATURES if c in df_2024.columns]

# Identify feature types
categorical_features = []
for cand in ['DayOfWeek','Month','DayOfMonth','Year','is_weekend','is_month_start','isWeekend','isMacauHoliday','isChinaHoliday','weekendHoliday']:
    if cand in BASELINE_FEATURES:
        categorical_features.append(cand)

numeric_features = [c for c in BASELINE_FEATURES if c not in categorical_features]

print("Baseline features:", BASELINE_FEATURES)
print("Categorical:", categorical_features)
print("Numeric:", numeric_features)


In [ ]:
# Time-based split within 2024: keep last 28 days as validation
assert not df_2024.empty, "No 2024 data available."

last_date_2024 = df_2024['FT_CalendarDate'].max()
val_days = 28
val_start = last_date_2024 - pd.Timedelta(days=val_days-1)

train_df = df_2024.loc[df_2024['FT_CalendarDate'] < val_start].copy()
val_df = df_2024.loc[(df_2024['FT_CalendarDate'] >= val_start) & (df_2024['FT_CalendarDate'] <= last_date_2024)].copy()

print("Train period:", train_df['FT_CalendarDate'].min(), '->', train_df['FT_CalendarDate'].max(), len(train_df))
print("Valid period:", val_df['FT_CalendarDate'].min(), '->', val_df['FT_CalendarDate'].max(), len(val_df))

X_train = train_df[BASELINE_FEATURES]
y_train = train_df[TARGET]
X_val = val_df[BASELINE_FEATURES]
y_val = val_df[TARGET]


In [ ]:
# Build baseline model pipeline (no event features)

preprocess = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features),
        ('num', StandardScaler(), numeric_features)
    ],
    remainder='drop'
)

model = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

pipeline = Pipeline(steps=[('prep', preprocess), ('model', model)])

pipeline.fit(X_train, y_train)

val_pred = pipeline.predict(X_val)
print({
    'MAE': mean_absolute_error(y_val, val_pred),
    'RMSE': rmse(y_val, val_pred),
    'sMAPE': smape(y_val, val_pred)
})


In [ ]:
# Residuals and large residual days
val_residuals = y_val.values - val_pred

# Robust threshold via MAD
residuals_abs = np.abs(val_residuals)
mad_val = mad(residuals_abs)
median_val = np.median(residuals_abs)
K = 3.0  # tunable
thr = median_val + K * mad_val

val_df = val_df.copy()
val_df['y_true'] = y_val.values
val_df['y_pred'] = val_pred
val_df['residual'] = val_residuals
val_df['abs_residual'] = residuals_abs
val_df['large_residual'] = val_df['abs_residual'] >= thr

print({
    'median_abs_residual': float(median_val),
    'mad_abs_residual': float(mad_val),
    'threshold': float(thr),
    'num_large_residual_days': int(val_df['large_residual'].sum()),
    'val_days': int(len(val_df))
})

display(val_df[['FT_CalendarDate','y_true','y_pred','residual','abs_residual','large_residual']].head(5))


In [ ]:
# Event alignment and tests

# Define event day flag (robust): true if EventSize_Category != 'None' or EventSize_MinMax > 0
if 'EventSize_Category' in val_df.columns:
    cat_flag = val_df['EventSize_Category'].astype(str).str.lower() != 'none'
else:
    cat_flag = pd.Series(False, index=val_df.index)

if 'EventSize_MinMax' in val_df.columns:
    num_flag = (val_df['EventSize_MinMax'].fillna(0) > 0)
else:
    num_flag = pd.Series(False, index=val_df.index)

val_df['event_day'] = (cat_flag | num_flag).astype(int)

# Contingency for Fisher test: large_residual vs event_day
A = int(((val_df['large_residual'] == True) & (val_df['event_day'] == 1)).sum())
B = int(((val_df['large_residual'] == True) & (val_df['event_day'] == 0)).sum())
C = int(((val_df['large_residual'] == False) & (val_df['event_day'] == 1)).sum())
D = int(((val_df['large_residual'] == False) & (val_df['event_day'] == 0)).sum())

print({'A_LR&Event': A, 'B_LR&NonEvent': B, 'C_NonLR&Event': C, 'D_NonLR&NonEvent': D})

if SCIPY_AVAILABLE:
    table = np.array([[A, B],[C, D]])
    oddsratio, pvalue = fisher_exact(table, alternative='greater')
    print({'fisher_oddsratio': float(oddsratio), 'fisher_pvalue': float(pvalue)})
else:
    print("SciPy not available: skipping Fisher's exact test.")

# AUC: use |residual| to discriminate event vs non-event
try:
    auc = roc_auc_score(val_df['event_day'], val_df['abs_residual'])
    print({'ROC_AUC_abs_residual_vs_event': float(auc)})
except Exception as e:
    print('AUC error:', e)

display(val_df[['FT_CalendarDate','event_day','large_residual','abs_residual']].head(10))


In [ ]:
# Visualization
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(val_df['FT_CalendarDate'], val_df['residual'], label='Residual', color='tab:blue')

# Highlight event days
evt_dates = val_df.loc[val_df['event_day'] == 1, 'FT_CalendarDate']
ax.scatter(evt_dates, np.zeros(len(evt_dates)), color='red', marker='|', s=200, label='Event day')

# Highlight large residual days
lr_dates = val_df.loc[val_df['large_residual'] == True, 'FT_CalendarDate']
ax.scatter(lr_dates, val_df.loc[val_df['large_residual'] == True, 'residual'], color='orange', label='Large residual')

ax.axhline(0, color='black', linewidth=1)
ax.set_title(f"Residuals with events — {ACTIVE_LOCATION if ACTIVE_LOCATION else 'All locations'}")
ax.set_xlabel('Date')
ax.set_ylabel('Residual (y - y_hat)')
ax.legend()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Multi-location analysis: per-location baseline, residuals, and event tests

def build_and_evaluate_per_location(df_all, year=2024, val_days=28, k_mad=3.0):
    results = []
    locs = sorted(df_all['Location'].dropna().unique())
    for loc in locs:
        d = df_all.loc[df_all['Location'] == loc].copy()
        d = d.sort_values('FT_CalendarDate')
        d_yr = d.loc[d['Year'] == year].copy()
        if d_yr.empty or d_yr['FT_CalendarDate'].nunique() < 40:
            continue
        # features
        base_feats = [
            'DayOfWeek','Month','isWeekend','isMacauHoliday','isChinaHoliday',
            'consecutiveChina','consecutiveMacau','DayOfMonth','Year','is_weekend',
            'is_month_start','days_to_holiday','holiday_weight','holiday_decay','weekendHoliday'
        ]
        base_feats = [c for c in base_feats if c in d_yr.columns]
        cat_feats = [c for c in ['DayOfWeek','Month','DayOfMonth','Year','is_weekend','is_month_start','isWeekend','isMacauHoliday','isChinaHoliday','weekendHoliday'] if c in base_feats]
        num_feats = [c for c in base_feats if c not in cat_feats]
        if len(base_feats) == 0:
            continue
        # split
        last_dt = d_yr['FT_CalendarDate'].max()
        val_start = last_dt - pd.Timedelta(days=val_days-1)
        tr = d_yr.loc[d_yr['FT_CalendarDate'] < val_start].copy()
        va = d_yr.loc[(d_yr['FT_CalendarDate'] >= val_start) & (d_yr['FT_CalendarDate'] <= last_dt)].copy()
        if tr.empty or va.empty:
            continue
        X_tr, y_tr = tr[base_feats], tr['Actual']
        X_va, y_va = va[base_feats], va['Actual']
        # model
        prep = ColumnTransformer([
            ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_feats),
            ('num', StandardScaler(), num_feats)
        ])
        mdl = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
        pipe = Pipeline([('prep', prep), ('model', mdl)])
        pipe.fit(X_tr, y_tr)
        pred = pipe.predict(X_va)
        # residuals
        res = y_va.values - pred
        abs_res = np.abs(res)
        m = mad(abs_res)
        med = np.median(abs_res)
        thr = med + k_mad * m
        large = abs_res >= thr
        # event day flag
        if 'EventSize_Category' in va.columns:
            cat_flag = va['EventSize_Category'].astype(str).str.lower() != 'none'
        else:
            cat_flag = pd.Series(False, index=va.index)
        if 'EventSize_MinMax' in va.columns:
            num_flag = (va['EventSize_MinMax'].fillna(0) > 0)
        else:
            num_flag = pd.Series(False, index=va.index)
        evt = (cat_flag | num_flag).astype(int).values
        # fisher
        A = int((large & (evt==1)).sum())
        B = int((large & (evt==0)).sum())
        C = int(((~large) & (evt==1)).sum())
        D = int(((~large) & (evt==0)).sum())
        fisher_p = None
        fisher_or = None
        if SCIPY_AVAILABLE:
            table = np.array([[A, B],[C, D]])
            try:
                fisher_or, fisher_p = fisher_exact(table, alternative='greater')
            except Exception:
                fisher_p = None
                fisher_or = None
        # auc
        auc = None
        try:
            auc = roc_auc_score(evt, abs_res)
        except Exception:
            auc = None
        results.append({
            'Location': loc,
            'TrainDays': int(len(tr)),
            'ValDays': int(len(va)),
            'MAE': float(mean_absolute_error(y_va, pred)),
            'RMSE': float(rmse(y_va, pred)),
            'sMAPE': float(smape(y_va, pred)),
            'MedianAbsRes': float(med),
            'MAD_AbsRes': float(m),
            'Threshold': float(thr),
            'NumLargeRes': int(large.sum()),
            'NumEventsInVal': int((evt==1).sum()),
            'A_LR&Event': A, 'B_LR&NonEvent': B, 'C_NonLR&Event': C, 'D_NonLR&NonEvent': D,
            'Fisher_OR': None if fisher_or is None else float(fisher_or),
            'Fisher_p': None if fisher_p is None else float(fisher_p),
            'AUC_absRes_vs_Event': None if auc is None else float(auc)
        })
    return pd.DataFrame(results)

multi_loc_summary = build_and_evaluate_per_location(raw_df, year=2024, val_days=28, k_mad=3.0)
display(multi_loc_summary.sort_values(['Fisher_p','AUC_absRes_vs_Event'], ascending=[True, False]).head(20))


In [ ]:
# Export original + predictions table for active location (2024)

# Refit using the earlier split context for ACTIVE_LOCATION
export_cols = [c for c in df_2024.columns]

# Mark split
train_df = train_df.copy()
train_df['Split'] = 'train'
val_df = val_df.copy()
val_df['Split'] = 'valid'
val_df['y_pred'] = val_pred
val_df['residual'] = val_df['y_true'] - val_df['y_pred']

combined = pd.concat([train_df, val_df], axis=0, ignore_index=True, sort=False)

# Order columns: keys + target + preds + residuals + features
key_cols = ['FT_CalendarDate','Location']
extra_cols = ['Split','y_pred','residual','large_residual','event_day']
ordered = [c for c in key_cols + ['Actual'] + extra_cols + BASELINE_FEATURES + EVENT_FEATURES if c in combined.columns]
combined = combined[ordered].sort_values(['Location','FT_CalendarDate']).reset_index(drop=True)

display(combined.head(10))

# Save
out_path = "/Users/katadriano/Documents/ml_test_space/predictions_2024_active_location.csv"
combined.to_csv(out_path, index=False)
print("Saved:", out_path)


In [1]:
df_spark = spark.read.format("delta").load("Tables/ML_LPOS_Daily_Covers_Silver_Featured")
df_spark = df_spark.cache()
df = df_spark.toPandas()

StatementMeta(, 024ef8b6-550b-4c21-b18b-2aa513acda5a, 3, Finished, Available, Finished)

In [2]:
# check df FT_CalendarDate min and max
print(df['FT_CalendarDate'].min())
print(df['FT_CalendarDate'].max())

StatementMeta(, 024ef8b6-550b-4c21-b18b-2aa513acda5a, 4, Finished, Available, Finished)

2023-01-01 00:00:00
2026-02-17 00:00:00


In [16]:
START_DT = "2024-01-01T00:00:00"
END_DT = "2025-08-01T00:00:00"
LOCATIONS = [
     "Chiu Chow - Pak Loh - GM",
            "Chuan Yu Ren Jia - GM",
            "Yamazato - OK",
            "Italian - Bombana - GM",
            "Room Service - GM",
            # "Italian - Terrazza - GM",
            "Okura Lobby Lounge - Nagomi - OK",
            "Market Street-Dragon Portuguese Cuisine - BW",
            "Putien - GM",
            "All Day Dining - Andaz - AD",
            "Waso - GM",
            # "Whisky Bar - Macallan - GM",
            "Taiwanese - Lugang - GM",

            # Back up
            "Saffron - BT",
            "Cafe de Paris - GM",
            "Poolside - Breeze Cafe - GM",

            # Pharse 0 
            'Bei Shan Lou - GM',
            'Butao Ramen - GM',
            'Cha Bei - GM',
            'Chinese - The Noodle Kitchen - GM',
            'Pangs Kitchen - GM',
            'Passion Cafe - GM',
            'Tam Chai Yu Chun - GM',
            'Teppanyaki - GM',
            'The Apron - GM',
            'Watami - GM',


            # Pharse 2
            "Cinema Concessions - GM",
            "Blossom Palace - GM",
            "Market Street - Du Hsiao Yueh - BW",
            "Wui Sik Fong - BW",
            'Whiskey Bar - SW',
            'Feng Wei Ju - SW',
            'Room Service - SW',
            'Lobby Lounge - SW',
            'Passion Cafe - SW',
            'Sensations - SW',
            'Chang Sheng Fang - SW',
            'Temptations - SW',

            # Pharse 3
            'Oasis - GM',
            'Food Court Admin - GM'
]

SELECTED_FEATURES = [
    'DayOfWeek',
    'Month',
    'isWeekend',
    'isMacauHoliday',
    'isChinaHoliday',
    'consecutiveChina',
    'consecutiveMacau',
    'DayOfMonth',
    'Year',
    'is_weekend',
    'is_month_start',
    'EventSize_MinMax',
    'EventSize_Category',
    'days_to_holiday',
    'holiday_weight',
    'holiday_decay',
    'weekendHoliday'
]

TARGET_COL = ["Actual"]

IDEX_LIST = ['FT_CalendarDate','Location']

StatementMeta(, 024ef8b6-550b-4c21-b18b-2aa513acda5a, 18, Finished, Available, Finished)

In [19]:
# Filter df where location in LOCATIONS and FT_CalendarDate >= START_DT and  FT_CalendarDate <= END_DT
LOCATIONS = [
     "Chiu Chow - Pak Loh - GM"]
filtered_df = df[(df['Location'].isin(LOCATIONS)) & (df['FT_CalendarDate'] >= START_DT) & (df['FT_CalendarDate'] <= END_DT)][IDEX_LIST + SELECTED_FEATURES+TARGET_COL]
display(filtered_df)

# filter filtered_df EventSize_Category <> 'None'
test_df = filtered_df[filtered_df['EventSize_Category'] != 'None']
display(test_df)


StatementMeta(, 024ef8b6-550b-4c21-b18b-2aa513acda5a, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 18e03e3f-e7f7-4e44-a9ae-974cf22fda2d)

SynapseWidget(Synapse.DataFrame, e24bfc00-15ee-4834-beca-b3ef705b4d6d)